配置 Datafeeder

In [1]:
from __future__ import print_function
import paddle
import paddle.fluid as fluid
import numpy
import math
import sys

读取txt文件的数据

In [2]:
# housing.data是一个txt文件，包含506行，14列数据，每列的数据含义如下
# CRIM    该镇的人均犯罪率                      连续值
# ZN      占地面积超过25,000平方呎的住宅用地比例  连续值
# INDUS   非零售商业用地比例 连续值
# CHAS    是否邻近 Charles River               离散值，1=邻近；0=不邻近
# NOX     一氧化氮浓度  连续值
# RM      每栋房屋的平均客房数                  连续值
# AGE     1940年之前建成的自用单位比例           连续值
# DIS     到波士顿5个就业中心的加权距离          连续值
# RAD     到径向公路的可达性指数                连续值
# TAX     全值财产税率                         连续值
# PTRATIO 学生与教师的比例                     连续值
# B       1000(BK - 0.63)^2，其中BK为黑人占比  连续值
# LSTAT   低收入人群占比                       连续值
# MEDV    同类房屋价格的中位数                  连续值

#为每列数据命名（housing.data中的数据集未含标题）
feature_names = [
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
    'PTRATIO', 'B', 'LSTAT', 'convert'
]
feature_num = len(feature_names) # 获取列数=14
data = numpy.fromfile('housing.data', sep=' ') # 从文件中读取原始数据，数据用空格分割
# data.shape[0] 是数据数量，共506*14=7084个数据
# // 为地板除运算，结果为不大于商的最大整数。
# 将读入的数据转换为 [506,14]的二维张量
data = data.reshape(data.shape[0] // feature_num, feature_num)
# axis=0 表示按列（取最大、最小、平均）data.shape[0] =506
maximums, minimums, avgs = data.max(axis=0), data.min(axis=0), data.sum(axis=0)/data.shape[0]
maximums

array([ 88.9762, 100.    ,  27.74  ,   1.    ,   0.871 ,   8.78  ,
       100.    ,  12.1265,  24.    , 711.    ,  22.    , 396.9   ,
        37.97  ,  50.    ])

In [4]:
sums=data.sum(axis=0)
sums

array([1.82844292e+03, 5.75000000e+03, 5.63521000e+03, 3.50000000e+01,
       2.80675700e+02, 3.18002500e+03, 3.46989000e+04, 1.92029160e+03,
       4.83200000e+03, 2.06568000e+05, 9.33850000e+03, 1.80477060e+05,
       6.40245000e+03, 1.14016000e+04])

In [5]:
avgs

array([3.61352356e+00, 1.13636364e+01, 1.11367787e+01, 6.91699605e-02,
       5.54695059e-01, 6.28463439e+00, 6.85749012e+01, 3.79504269e+00,
       9.54940711e+00, 4.08237154e+02, 1.84555336e+01, 3.56674032e+02,
       1.26530632e+01, 2.25328063e+01])

In [6]:
data.shape[0]

506

数据归一化处理
减掉均值，然后除以原取值范围

In [7]:
for i in six.moves.range(feature_num-1):
   data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i]) # six.moves可以兼容python2和python3

NameError: name 'six' is not defined

In [8]:
data

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 3.9690e+02, 4.9800e+00,
        2.4000e+01],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 3.9690e+02, 9.1400e+00,
        2.1600e+01],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 3.9283e+02, 4.0300e+00,
        3.4700e+01],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 5.6400e+00,
        2.3900e+01],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 3.9345e+02, 6.4800e+00,
        2.2000e+01],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 7.8800e+00,
        1.1900e+01]])